In [1]:
import pandas as pd
import numpy as np
import re

## Обработка таблицы, совмещение с ID

In [2]:
# отчет

detalization_report = pd.read_excel('ИСХОДНИК_Отчет_1_детализация_02122019_08122019.xlsx')
# detalization_report.head(2)
detalization_report = detalization_report.fillna(0)
# detalization_report.info()

In [3]:
# проверка
detalization_report[(detalization_report['АПЦ'] == 'ГБУЗ ГП 23 ДЗМ') & (detalization_report['Специальность врача'] == 'Врач-кардиолог')].sort_values(by=['ФИЛИАЛ'])

,Обслуживаемое население,АПЦ,ФИЛИАЛ,Специальность врача,Количество слотов для записи в расписании всего,с шагом сетки < 12 минут,с шагом сетки 12-14 минут,с шагом сетки 15 минут,с шагом сетки 16-20 минут,с шагом сетки > 20 минут
3468,Взрослое,ГБУЗ ГП 23 ДЗМ,ГБУЗ ГП 23 ДЗМ,Врач-кардиолог,408,0.0,0.0,294.0,114.0,0.0
3023,Взрослое,ГБУЗ ГП 23 ДЗМ,ГБУЗ ГП 23 Ф 1,Врач-кардиолог,148,0.0,0.0,148.0,0.0,0.0
597,Взрослое,ГБУЗ ГП 23 ДЗМ,ГБУЗ ГП 23 Ф 2,Врач-кардиолог,148,0.0,0.0,148.0,0.0,0.0
1088,Взрослое,ГБУЗ ГП 23 ДЗМ,ГБУЗ ГП 23 Ф 3,Врач-кардиолог,82,0.0,0.0,82.0,0.0,0.0
2754,Взрослое,ГБУЗ ГП 23 ДЗМ,ГБУЗ ГП 23 Ф 4,Врач-кардиолог,60,0.0,0.0,60.0,0.0,0.0
1352,Смешанное,ГБУЗ ГП 23 ДЗМ,ГБУЗ ГП 23 Ф 5,Врач-кардиолог,88,0.0,0.0,88.0,0.0,0.0


In [4]:
full_ids = pd.read_excel('id.xlsx', header = 1)
# full_ids.info()

In [5]:
full_ids = full_ids.fillna(0)
# full_ids.info()

In [6]:
b = full_ids.select_dtypes(include=['float64']).columns
full_ids[b] = full_ids[b].astype(int)
# full_ids.info()

In [7]:
# ИД к заглавному регистру:
full_ids['ФИЛИАЛ'] = full_ids['ФИЛ (исходное)'].str.upper()
detalization_report['ФИЛИАЛ'] = detalization_report['ФИЛИАЛ'].str.upper()

In [8]:
full_df = pd.merge(full_ids, detalization_report, how='inner', left_on='ФИЛИАЛ', right_on='ФИЛИАЛ')

# значения дублируются из-за столбца 'АЦ (исходное)' - удаляю:
full_df = full_df.drop('АЦ (исходное)', 1) 

In [9]:
full_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4280 entries, 0 to 4279
Data columns (total 24 columns):
ФИЛ (исходное)                                     4280 non-null object
Округ                                              4280 non-null object
ФИЛ (полное)                                       4280 non-null object
Тип АЦ                                             4280 non-null object
АЦ (полное)                                        4280 non-null object
АЦ                                                 4280 non-null object
ID (рабочий)                                       4280 non-null object
id (с всп)                                         4280 non-null object
ГЗ/Ф                                               4280 non-null object
Прикрепленное население                            4280 non-null object
ИД_Ф_ЕМИАС                                         4280 non-null object
ИД_Ф                                               4280 non-null object
ИД_Ф_ЕМИАС.1                 

## Шаг сетки:

In [10]:
full_df['с шагом сетки < 15 минут'] = full_df['с шагом сетки < 12 минут']\
+ full_df['с шагом сетки 12-14 минут']

## Правила для врачей

In [11]:
doc_list = full_df['Специальность врача'].unique()    
len(doc_list)

22

In [12]:
doc_table = pd.read_excel('id.xlsx', sheet_name = 2, index_col='№')
# doc_table = doc_table.drop('Unnamed: 0', 1)
doc_table

,исходник,уровень,запись,обслуживаемое население
№,,,,
1,Врач-терапевт участковый,1,самозапись,Взрослое
2,Врач общей практики (семейный врач),1,самозапись,Взрослое
3,Врач-педиатр участковый,1,самозапись,Детское
4,Врач-хирург,1,самозапись,Взрослое
5,Врач - детский хирург,1,самозапись,Детское
6,Врач-оториноларинголог,1,самозапись,Смешанное
7,Врач-офтальмолог,1,самозапись,Смешанное
8,Врач-уролог,1,самозапись,Взрослое
9,Врач-травматолог-ортопед,1,направление,Детское


In [13]:
# создаю стандарт для взрослых

s1_ad = doc_table.loc[(doc_table['уровень'] == 1) & (doc_table['обслуживаемое население'] != 'Детское')]
s2_ad = doc_table.loc[(doc_table['уровень'] == 2) & (doc_table['обслуживаемое население'] != 'Детское')]

In [14]:
s1_ad

,исходник,уровень,запись,обслуживаемое население
№,,,,
1,Врач-терапевт участковый,1,самозапись,Взрослое
2,Врач общей практики (семейный врач),1,самозапись,Взрослое
4,Врач-хирург,1,самозапись,Взрослое
6,Врач-оториноларинголог,1,самозапись,Смешанное
7,Врач-офтальмолог,1,самозапись,Смешанное
8,Врач-уролог,1,самозапись,Взрослое
10,Врач-кардиолог,1,направление,Взрослое
11,Врач-невролог,1,направление,Смешанное
12,Врач-эндокринолог,1,направление,Взрослое


In [15]:
list(s2_ad['исходник'])

['Врач-аллерголог-иммунолог',
 'Врач-гастроэнтеролог',
 'Врач-инфекционист',
 'Врач-колопроктолог',
 'Врач-пульмонолог']

In [16]:
full_df = pd.merge(full_df, doc_table, how='inner', left_on='Специальность врача', right_on='исходник',  left_index=False)

In [17]:
full_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4280 entries, 0 to 4279
Data columns (total 29 columns):
ФИЛ (исходное)                                     4280 non-null object
Округ                                              4280 non-null object
ФИЛ (полное)                                       4280 non-null object
Тип АЦ                                             4280 non-null object
АЦ (полное)                                        4280 non-null object
АЦ                                                 4280 non-null object
ID (рабочий)                                       4280 non-null object
id (с всп)                                         4280 non-null object
ГЗ/Ф                                               4280 non-null object
Прикрепленное население                            4280 non-null object
ИД_Ф_ЕМИАС                                         4280 non-null object
ИД_Ф                                               4280 non-null object
ИД_Ф_ЕМИАС.1                 

In [18]:
test = full_df[(full_df['АЦ'] == 'ГП 23') & (full_df['исходник'] == 'Врач-кардиолог')].sort_values(by=['ID (рабочий)'])
test

,ФИЛ (исходное),Округ,ФИЛ (полное),Тип АЦ,АЦ (полное),АЦ,ID (рабочий),id (с всп),ГЗ/Ф,Прикрепленное население,...,с шагом сетки < 12 минут,с шагом сетки 12-14 минут,с шагом сетки 15 минут,с шагом сетки 16-20 минут,с шагом сетки > 20 минут,с шагом сетки < 15 минут,исходник,уровень,запись,обслуживаемое население
1618,ГБУЗ ГП 23 ДЗМ,ЮВАО,ГП № 23,АЦ взрослый,ГП № 23,ГП 23,ГП 23,ГП 23,ГЗ,58672,...,0.0,0.0,294.0,114.0,0.0,0.0,Врач-кардиолог,1,направление,Взрослое
1620,ГБУЗ ГП 23 Ф 1,ЮВАО,ГП № 23 филиал № 1,АЦ взрослый,ГП № 23,ГП 23,ГП 23 Ф 1,ГП 23 Ф 1,Ф,29039,...,0.0,0.0,148.0,0.0,0.0,0.0,Врач-кардиолог,1,направление,Взрослое
1621,ГБУЗ ГП 23 Ф 2,ЮВАО,ГП № 23 филиал № 2,АЦ взрослый,ГП № 23,ГП 23,ГП 23 Ф 2,ГП 23 Ф 2,Ф,45783,...,0.0,0.0,148.0,0.0,0.0,0.0,Врач-кардиолог,1,направление,Взрослое
1622,ГБУЗ ГП 23 Ф 3,ЮВАО,ГП № 23 филиал № 3,АЦ взрослый,ГП № 23,ГП 23,ГП 23 Ф 3,ГП 23 Ф 3,Ф,38346,...,0.0,0.0,82.0,0.0,0.0,0.0,Врач-кардиолог,1,направление,Взрослое
1623,ГБУЗ ГП 23 Ф 4,ЮВАО,ГП № 23 филиал № 4,АЦ взрослый,ГП № 23,ГП 23,ГП 23 Ф 4,ГП 23 Ф 4,Ф,21829,...,0.0,0.0,60.0,0.0,0.0,0.0,Врач-кардиолог,1,направление,Взрослое
1619,ГБУЗ ГП 23 Ф 5,ЮВАО,ГП № 23 филиал № 5,АЦ взрослый,ГП № 23,ГП 23,ГП 23 Ф 5,ГП 23 Ф 5,Ф,31880,...,0.0,0.0,88.0,0.0,0.0,0.0,Врач-кардиолог,1,направление,Взрослое



### Надо собрать регулярку которая добавить название АЦ

In [19]:
# НЕ ПОМНЮ ЗАЧЕМ)) 

# def make_AC(row):
#     try:
#         result = re.search(r'^\w+\s\w+', row).group(0)
#     except AttributeError:
#         result = 0
    
#     return result

# full_df['AC'] = full_df['id'].apply(make_AC)

## Pivot

In [20]:
full_df.head(2)

,ФИЛ (исходное),Округ,ФИЛ (полное),Тип АЦ,АЦ (полное),АЦ,ID (рабочий),id (с всп),ГЗ/Ф,Прикрепленное население,...,с шагом сетки < 12 минут,с шагом сетки 12-14 минут,с шагом сетки 15 минут,с шагом сетки 16-20 минут,с шагом сетки > 20 минут,с шагом сетки < 15 минут,исходник,уровень,запись,обслуживаемое население
0,ГБ Кузнечики амбулатория Кленово,ТиНАО,Больница «Кузнечики» (амбулатория Кленово),ПО,Больница «Кузнечики»,Больница «Кузнечики» (амб. Кленово),Больница «Кузнечики» (амб. Кленово),Больница «Кузнечики» (амб. Кленово),ГЗ,0,...,0.0,0.0,24.0,0.0,0.0,0.0,Врач - детский эндокринолог,2,направление,Детское
1,ГБ Кузнечики амбулатория Кленово,ТиНАО,Больница «Кузнечики» (амбулатория Кленово),ПО,Больница «Кузнечики»,Больница «Кузнечики» (амб. Кленово),Больница «Кузнечики» (амб. Кленово),Больница «Кузнечики» (амб. Кленово),ГЗ,0,...,0.0,0.0,24.0,0.0,0.0,0.0,Врач - детский эндокринолог,2,направление,Детское


In [21]:
list(full_df.columns)

['ФИЛ (исходное)',
 'Округ',
 'ФИЛ (полное)',
 'Тип АЦ',
 'АЦ (полное)',
 'АЦ',
 'ID (рабочий)',
 'id (с всп)',
 'ГЗ/Ф',
 'Прикрепленное население',
 'ИД_Ф_ЕМИАС',
 'ИД_Ф',
 'ИД_Ф_ЕМИАС.1',
 'ИД_Ф.1',
 'ФИЛИАЛ',
 'Обслуживаемое население',
 'АПЦ',
 'Специальность врача',
 'Количество слотов для записи в расписании всего',
 'с шагом сетки < 12 минут',
 'с шагом сетки 12-14 минут',
 'с шагом сетки 15 минут',
 'с шагом сетки 16-20 минут',
 'с шагом сетки > 20 минут',
 'с шагом сетки < 15 минут',
 'исходник',
 'уровень',
 'запись',
 'обслуживаемое население']

In [22]:
test_pivot = pd.pivot_table(full_df,
                            index=['Округ','АЦ','ID (рабочий)','Тип АЦ','Прикрепленное население'], 
                            columns=['Специальность врача'],
                            values='Количество слотов для записи в расписании всего',
                            
                            fill_value=0)
                            


test_pivot.reset_index(inplace = True)
test_pivot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 443 entries, 0 to 442
Data columns (total 27 columns):
Округ                                  443 non-null object
АЦ                                     443 non-null object
ID (рабочий)                           443 non-null object
Тип АЦ                                 443 non-null object
Прикрепленное население                443 non-null object
Врач - детский кардиолог               443 non-null int64
Врач - детский уролог-андролог         443 non-null int64
Врач - детский хирург                  443 non-null int64
Врач - детский эндокринолог            443 non-null int64
Врач общей практики (семейный врач)    443 non-null float64
Врач-акушер-гинеколог                  443 non-null float64
Врач-аллерголог-иммунолог              443 non-null int64
Врач-гастроэнтеролог                   443 non-null int64
Врач-инфекционист                      443 non-null int64
Врач-кардиолог                         443 non-null int64
Врач-колопроктол

In [23]:
len(test_pivot.columns)

27

In [24]:
test_pivot.columns[4:]

Index(['Прикрепленное население', 'Врач - детский кардиолог',
       'Врач - детский уролог-андролог', 'Врач - детский хирург',
       'Врач - детский эндокринолог', 'Врач общей практики (семейный врач)',
       'Врач-акушер-гинеколог', 'Врач-аллерголог-иммунолог',
       'Врач-гастроэнтеролог', 'Врач-инфекционист', 'Врач-кардиолог',
       'Врач-колопроктолог', 'Врач-невролог', 'Врач-нефролог',
       'Врач-оториноларинголог', 'Врач-офтальмолог', 'Врач-педиатр участковый',
       'Врач-пульмонолог', 'Врач-терапевт участковый',
       'Врач-травматолог-ортопед', 'Врач-уролог', 'Врач-хирург',
       'Врач-эндокринолог'],
      dtype='object', name='Специальность врача')

In [25]:
adult_df = test_pivot[test_pivot['Тип АЦ'] == 'АЦ взрослый'] 

In [26]:
adult_df.head(3)

Специальность врача,Округ,АЦ,ID (рабочий),Тип АЦ,Прикрепленное население,Врач - детский кардиолог,Врач - детский уролог-андролог,Врач - детский хирург,Врач - детский эндокринолог,Врач общей практики (семейный врач),...,Врач-нефролог,Врач-оториноларинголог,Врач-офтальмолог,Врач-педиатр участковый,Врач-пульмонолог,Врач-терапевт участковый,Врач-травматолог-ортопед,Врач-уролог,Врач-хирург,Врач-эндокринолог
2,ВАО,ГП 175,ГП 175,АЦ взрослый,35496,0,0,0,0,426.0,...,0,169.0,297.0,0,0,849,0,162,180.0,134
3,ВАО,ГП 175,ГП 175 Ф 1,АЦ взрослый,57097,0,0,0,0,694.0,...,0,169.0,194.0,0,0,1698,0,284,344.0,192
4,ВАО,ГП 175,ГП 175 Ф 2,АЦ взрослый,45716,0,0,0,0,509.0,...,0,239.0,254.0,0,0,1066,272,201,260.0,140


In [27]:
# приводим все к `int`
_ = adult_df.select_dtypes(include=['float64']).columns
adult_df[_] = adult_df[_].astype(int)

# удаляем лишний столбец
adult_df = adult_df.drop('Тип АЦ', 1) 

adult_df.head(3)

C:\Users\Asus\Anaconda3\lib\site-packages\pandas\core\frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


Специальность врача,Округ,АЦ,ID (рабочий),Прикрепленное население,Врач - детский кардиолог,Врач - детский уролог-андролог,Врач - детский хирург,Врач - детский эндокринолог,Врач общей практики (семейный врач),Врач-акушер-гинеколог,...,Врач-нефролог,Врач-оториноларинголог,Врач-офтальмолог,Врач-педиатр участковый,Врач-пульмонолог,Врач-терапевт участковый,Врач-травматолог-ортопед,Врач-уролог,Врач-хирург,Врач-эндокринолог
2,ВАО,ГП 175,ГП 175,35496,0,0,0,0,426,0,...,0,169,297,0,0,849,0,162,180,134
3,ВАО,ГП 175,ГП 175 Ф 1,57097,0,0,0,0,694,0,...,0,169,194,0,0,1698,0,284,344,192
4,ВАО,ГП 175,ГП 175 Ф 2,45716,0,0,0,0,509,0,...,0,239,254,0,0,1066,272,201,260,140


### Функция для создания отчета:

In [68]:
def make_report(df, docs_list):
    new = df.iloc[:,:4].reset_index().drop(columns=['index'])
    for d in docs_list:
        if d in list(df.columns):
            new[d] = df[d].values
    return new

In [183]:
ad_s1 = make_report(adult_df, list(s1_ad['исходник']))

In [184]:
ad_s1['Участковый терапевт'] = ad_s1['Врач-терапевт участковый'] + ad_s1['Врач общей практики (семейный врач)']

In [185]:
ad_s1 = ad_s1[['Округ',
'АЦ',
'ID (рабочий)',
'Прикрепленное население',
'Участковый терапевт',
'Врач-терапевт участковый',
'Врач общей практики (семейный врач)',
'Врач-хирург',
'Врач-оториноларинголог',
'Врач-офтальмолог',
'Врач-уролог',
'Врач-кардиолог',
'Врач-невролог',
'Врач-эндокринолог']]

In [186]:
ad_s1.head(3)

Специальность врача,Округ,АЦ,ID (рабочий),Прикрепленное население,Участковый терапевт,Врач-терапевт участковый,Врач общей практики (семейный врач),Врач-хирург,Врач-оториноларинголог,Врач-офтальмолог,Врач-уролог,Врач-кардиолог,Врач-невролог,Врач-эндокринолог
0,ВАО,ГП 175,ГП 175,35496,1275,849,426,180,169,297,162,98,120,134
1,ВАО,ГП 175,ГП 175 Ф 1,57097,2392,1698,694,344,169,194,284,118,120,192
2,ВАО,ГП 175,ГП 175 Ф 2,45716,1575,1066,509,260,239,254,201,90,144,140


### Функция которая добавляет столбец 'Нарушение стандарта врачей 1 уровня'

In [190]:
def checker_s1(row):
    """Нарушение стандарта врачей 1 уровня"""
    if row == 0:
        return 'есть'
    else: return ''

In [191]:
# врачи по которым смотрим "0"

final_docs_list_for_s1 = [
'Участковый терапевт',
'Врач-хирург',
'Врач-оториноларинголог',
'Врач-офтальмолог',
'Врач-уролог',
'Врач-кардиолог',
'Врач-невролог',
'Врач-эндокринолог']

In [192]:
# можно упростить
# здесь появляется столбец с ошибкой по каждому врачу
for _ in final_docs_list_for_s1:
    ad_s1[f'{_}_error'] = ad_s1[_].apply(checker_s1)
    
ad_s1.sample(7)

Специальность врача,Округ,АЦ,ID (рабочий),Прикрепленное население,Участковый терапевт,Врач-терапевт участковый,Врач общей практики (семейный врач),Врач-хирург,Врач-оториноларинголог,Врач-офтальмолог,...,Врач-невролог,Врач-эндокринолог,Участковый терапевт_error,Врач-хирург_error,Врач-оториноларинголог_error,Врач-офтальмолог_error,Врач-уролог_error,Врач-кардиолог_error,Врач-невролог_error,Врач-эндокринолог_error
111,СЗАО,ГП 180,ГП 180 Ф 3,58293,2573,393,2180,426,355,345,...,290,321,,,,,,,,
12,ВАО,ГП 64,ГП 64 Ф 1,30962,2635,30,2605,390,76,210,...,92,87,,,,,,,,
63,САО,ГП 6,ГП 6 Ф 1,27901,1501,0,1501,184,185,185,...,144,120,,,,,,есть,,
173,ЮВАО,ГП 19,ГП 19 Ф 2,45996,2149,1002,1147,168,286,179,...,168,179,,,,,,есть,,
141,ЮАО,ГП 166,ГП 166 Ф 1,38600,1692,103,1589,343,93,275,...,697,171,,,,,,,,
122,ЦАО,ГП 3,ГП 3 Ф 1,20235,2432,1700,732,96,48,120,...,120,120,,,,,,,,
6,ВАО,ГП 191,ГП 191,64604,2672,1557,1115,728,478,419,...,513,334,,,,,,,,


In [194]:
# # добавляю столбец с суммой по ошибкам "естьестьесть"
# это надо упаковать в функцию
ad_s1['Нарушение стандарта врачей 1 уровня'] = ad_s1.iloc[:,14:22].apply(np.sum, axis=1)

In [195]:
# # удаляю лишние столбцы
cols = list(range(14,22))
ad_s1.drop(ad_s1.columns[cols],axis=1,inplace=True)

In [196]:
ad_s1[ad_s1['Нарушение стандарта врачей 1 уровня'] == 'естьестьестьесть']

Специальность врача,Округ,АЦ,ID (рабочий),Прикрепленное население,Участковый терапевт,Врач-терапевт участковый,Врач общей практики (семейный врач),Врач-хирург,Врач-оториноларинголог,Врач-офтальмолог,Врач-уролог,Врач-кардиолог,Врач-невролог,Врач-эндокринолог,Нарушение стандарта врачей 1 уровня
56,САО,ГП 45,ГП 45,нет прикрепленки,53,0,53,294,263,0,0,78,0,0,естьестьестьесть


In [197]:
ad_s1 = ad_s1[['Округ',
'АЦ',
'ID (рабочий)',
'Прикрепленное население',
'Нарушение стандарта врачей 1 уровня',
'Участковый терапевт',
'Врач-терапевт участковый',
'Врач общей практики (семейный врач)',
'Врач-хирург',
'Врач-оториноларинголог',
'Врач-офтальмолог',
'Врач-уролог',
'Врач-кардиолог',
'Врач-невролог',
'Врач-эндокринолог']]

In [198]:
ad_s1.sample(8)

Специальность врача,Округ,АЦ,ID (рабочий),Прикрепленное население,Нарушение стандарта врачей 1 уровня,Участковый терапевт,Врач-терапевт участковый,Врач общей практики (семейный врач),Врач-хирург,Врач-оториноларинголог,Врач-офтальмолог,Врач-уролог,Врач-кардиолог,Врач-невролог,Врач-эндокринолог
110,СЗАО,ГП 180,ГП 180 Ф 2,54604,,3170,1012,2158,405,270,355,267,140,270,310
146,ЮАО,ГП 170,ГП 170 Ф 2,29593,есть,1457,224,1233,350,175,175,169,105,133,0
8,ВАО,ГП 191,ГП 191 Ф 2,43500,,3629,3091,538,280,262,163,171,146,99,152
18,ВАО,ГП 66,ГП 66 Ф 4,44890,,2080,522,1558,350,175,235,208,119,182,112
157,ЮАО,ГП 214,ГП 214,80221,,2756,1704,1052,402,177,309,299,150,368,218
164,ЮАО,ГП 67,ГП 67,14598,естьестьесть,2315,320,1995,187,0,151,0,0,26,78
95,СВАО,ГП 218,ГП 218 Ф 5,32747,,1239,472,767,496,180,128,179,104,109,88
139,ЦАО,ГП 68,ГП 68 Ф 4,17446,есть,433,0,433,165,72,137,69,62,73,0


In [40]:
# по идее более простой способ, но здесь ошибка
# ad_s1['Нарушение стандарта врачей 1 уровня'] = ad_s1.apply(watcher, axis=1)

#### еще надо добавить автоматический подсчет для нарушений

### стандарт 1 для взрослых, автоматический подсчет. Необходимо добавить шапку и форматриование

In [41]:
ad_s1.to_excel("ad_s1.xlsx")

# ДАЛЕЕ

In [42]:
list(s2_ad['исходник'])

['Врач-аллерголог-иммунолог',
 'Врач-гастроэнтеролог',
 'Врач-инфекционист',
 'Врач-колопроктолог',
 'Врач-пульмонолог']

In [43]:
adult_df.head()

Специальность врача,Округ,АЦ,ID (рабочий),Прикрепленное население,Врач - детский кардиолог,Врач - детский уролог-андролог,Врач - детский хирург,Врач - детский эндокринолог,Врач общей практики (семейный врач),Врач-акушер-гинеколог,...,Врач-нефролог,Врач-оториноларинголог,Врач-офтальмолог,Врач-педиатр участковый,Врач-пульмонолог,Врач-терапевт участковый,Врач-травматолог-ортопед,Врач-уролог,Врач-хирург,Врач-эндокринолог
2,ВАО,ГП 175,ГП 175,35496,0,0,0,0,426,0,...,0,169,297,0,0,849,0,162,180,134
3,ВАО,ГП 175,ГП 175 Ф 1,57097,0,0,0,0,694,0,...,0,169,194,0,0,1698,0,284,344,192
4,ВАО,ГП 175,ГП 175 Ф 2,45716,0,0,0,0,509,0,...,0,239,254,0,0,1066,272,201,260,140
5,ВАО,ГП 175,ГП 175 Ф 3,46528,0,0,0,0,1031,0,...,0,134,60,0,0,486,0,169,120,190
6,ВАО,ГП 175,ГП 175 Ф 4,38139,0,0,0,0,894,0,...,0,158,150,0,0,639,0,126,208,168


In [44]:
list(s2_ad['исходник'])

['Врач-аллерголог-иммунолог',
 'Врач-гастроэнтеролог',
 'Врач-инфекционист',
 'Врач-колопроктолог',
 'Врач-пульмонолог']

In [45]:
ad_s2 = make_report(adult_df, list(s2_ad['исходник']))

In [46]:
ad_s2.head(3)

Специальность врача,Округ,АЦ,ID (рабочий),Прикрепленное население,Врач-аллерголог-иммунолог,Врач-гастроэнтеролог,Врач-инфекционист,Врач-колопроктолог,Врач-пульмонолог
0,ВАО,ГП 175,ГП 175,35496,0,0,0,0,0
1,ВАО,ГП 175,ГП 175 Ф 1,57097,0,0,0,0,0
2,ВАО,ГП 175,ГП 175 Ф 2,45716,0,0,0,0,0


### только ГЗ

In [47]:
ad_s2_only_H = ad_s2[ad_s2['АЦ'] == ad_s2['ID (рабочий)']]
ad_s2_only_H.head(3)

Специальность врача,Округ,АЦ,ID (рабочий),Прикрепленное население,Врач-аллерголог-иммунолог,Врач-гастроэнтеролог,Врач-инфекционист,Врач-колопроктолог,Врач-пульмонолог
0,ВАО,ГП 175,ГП 175,35496,0,0,0,0,0
6,ВАО,ГП 191,ГП 191,64604,0,180,108,0,110
11,ВАО,ГП 64,ГП 64,69041,0,0,112,0,74


In [48]:
def H_checker_s2(param):
    """Нарушение стандарта врачей 2 уровня в ГЗ"""
    if param == 0:
        return 'не принимал в ГЗ, '
    else: return ''

In [49]:
# можно упростить
# здесь появляется столбец с ошибкой по каждому врачу
for _ in list(s2_ad['исходник']):
    ad_s2_only_H[f'{_}_error'] = ad_s2_only_H[_].apply(H_checker_s2)
    
# # добавляю столбец с суммой по ошибкам "естьестьесть"
ad_s2_only_H['Специалист не принимал в ГЗ'] = ad_s2_only_H.iloc[:,10:15].apply(np.sum, axis=1)

# # удаляю лишние столбцы
cols = list(range(10,15))
ad_s2_only_H.drop(ad_s2_only_H.columns[cols],axis=1,inplace=True)

C:\Users\Asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\Asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\Asus\Anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-

In [50]:
print(ad_s2_only_H.shape)
ad_s2_only_H.head(2)

(46, 10)


Специальность врача,Округ,АЦ,ID (рабочий),Прикрепленное население,Врач-аллерголог-иммунолог,Врач-гастроэнтеролог,Врач-инфекционист,Врач-колопроктолог,Врач-пульмонолог,Врач-аллерголог-иммунолог_error
0,ВАО,ГП 175,ГП 175,35496,0,0,0,0,0,"не принимал в ГЗ,"
6,ВАО,ГП 191,ГП 191,64604,0,180,108,0,110,"не принимал в ГЗ,"


### только Филиалы

In [51]:
ad_s2_only_F = ad_s2[ad_s2['ID (рабочий)'].str.contains('Ф')==True]
ad_s2_only_F.head(2)

Специальность врача,Округ,АЦ,ID (рабочий),Прикрепленное население,Врач-аллерголог-иммунолог,Врач-гастроэнтеролог,Врач-инфекционист,Врач-колопроктолог,Врач-пульмонолог
1,ВАО,ГП 175,ГП 175 Ф 1,57097,0,0,0,0,0
2,ВАО,ГП 175,ГП 175 Ф 2,45716,0,0,0,0,0


In [52]:
def F_checker_s2(param):
    """Нарушение стандарта врачей 1 уровня"""
    if param != 0:
        return 1
    else: return ''

In [53]:
# можно упростить
# здесь появляется столбец с ошибкой по каждому врачу
for _ in list(s2_ad['исходник']):
    ad_s2_only_F[f'{_}_error'] = ad_s2_only_F[_].apply(F_checker_s2)

C:\Users\Asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [54]:
cols = list(range(9,14))
ad_s2_only_F.columns[cols]
ad_s2_only_F[ad_s2_only_F.columns[cols]].sum

<bound method DataFrame.sum of Специальность врача Врач-аллерголог-иммунолог_error  \
1                                                     
2                                                     
3                                                     
4                                                     
5                                                     
7                                                     
8                                                     
9                                                     
10                                                    
12                                                    
13                                                    
15                                                    
16                                                    
17                                                    
18                                                    
20                                                    
21                                

In [55]:

    
# # добавляю столбец с суммой по ошибкам
ad_s2_only_F['Специалист вел прием в филиале'] = ad_s2_only_F[ad_s2_only_F.iloc[cols]].sum

# # удаляю лишние столбцы
ad_s2_only_F.drop(ad_s2_only_F.columns[cols],axis=1,inplace=True)

ValueError: Must pass DataFrame with boolean values only

In [ ]:
print(ad_s2_only_F.shape)
ad_s2_only_F['Специалист вел прием в филиале'].unique()

In [ ]:
ad_s2_only_F_2 = ad_s2_only_F.groupby(['АЦ']).sum()
ad_s2_only_F_2

### Собрал стандарт 2 для взрослых. Необходимо добавить шапку, вывод, нарушения, несоответствия специалистов стандартам.

In [ ]:
adult_1_df = adult_df.groupby(['ac_id']).sum()

In [ ]:
adult_1_df.head(2)

In [ ]:
new = pd.DataFrame(ad_s2.index)

In [ ]:
doc_1_standart

In [ ]:
list(adult_1_df.columns)

In [ ]:
adult_1_df['Врач-уролог'].values

In [ ]:
for d in doc_2_standart:
    if d in list(ad_s2.columns):
        new[d] = ad_s2[d].values

In [ ]:
new

Написать код для текстового опредления отклонения по стандартам приема врачей.

In [ ]:
ad_s2